In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset_path = '/content/drive/MyDrive/Datasets/fake_news'
train = pd.read_csv(f'{dataset_path}/train.csv')
test = pd.read_csv(f'{dataset_path}/test.csv')
print(train.shape, train.head())
print(test.shape, test.head())

(20800, 5)    id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  
(5200, 4)       id                                              title  \
0  20800  Specter of Trump Loosens Tongues, if Not Purse...   


In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
id          0
title     122
author    503
text        7
dtype: int64


In [ ]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)
print(train.isnull().sum())
print(test.isnull().sum())

id        0
title     0
author    0
text      0
label     0
dtype: int64
id        0
title     0
author    0
text      0
dtype: int64


In [ ]:
test['content'] = test['author'] + test['title'] + test['text']
train['content'] = train['author'] + train['title'] + train['text']

# x_train, x_test, y_train, y_test = train['content'], test['content'], train['label'], test['label']

In [ ]:
def lemmatize_text(text):
  lemmetizer = WordNetLemmatizer()
  text = re.sub('^a-zA-Z', ' ', text)
  text = text.lower()
  words = text.split()
  words = [lemmetizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
  return ' '.join(words)

test['content'].apply(lemmatize_text)

,content
0,"david streitfeldspecter trump loosens tongues,..."
1,russian warship ready strike terrorist near al...
2,common dreams#nodapl: native american leader v...
3,daniel victortim tebow attempt another comebac...
4,truth broadcast networkkeiser report: meme war...
...,...
5195,jody rosenthe bangladeshi traffic jam never en...
5196,sheryl gay stolbergjohn kasich sign one aborti...
5197,"mike mcphatecalifornia today: what, exactly, s..."
5198,300 u marine deployed russian border norway« p...


In [ ]:
X = train['content'].values
y = train['label'].values
print(X.shape, y.shape)

(20800,) (20800,)


In [ ]:
vectorise = TfidfVectorizer()
X = vectorise.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred_train = model.predict(X_train)
print(accuracy_score(y_train, y_pred_train))
y_pred_test = model.predict(X_test)
print(accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

0.9739783653846154
0.9463942307692308
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      2082
           1       0.94      0.96      0.95      2078

    accuracy                           0.95      4160
   macro avg       0.95      0.95      0.95      4160
weighted avg       0.95      0.95      0.95      4160

